# Assignment 6
In this assignment, I import data from the San Francisco development pipeline from SF Open data's API. I then explore residential development that is proposed or currently under construction in San Francisco in a blog post. At the end of the notebook, I include a link to the blog post. 


First, import the packages necessary for calling an API. 

In [ ]:
#import packages
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import re as re
import json    # library for working with JSON-formatted text strings
import requests  # library for accessing content from web URLs
import pprint  # library for making Python data structures readable
pp = pprint.PrettyPrinter()

The SF Planning Department releases this data quarterly. Quarterly reports go back all the way to 2012. However, this data is messy and inconsistent. Therefore, for the purposes of this assignment, I have preserved some of the code in comments 

In [ ]:
Q22016 = 'https://data.sfgov.org/resource/3n2r-nn4r.json'

#Note as of (10/07/2016): holding off on importing these due to complications. 
#Some of the files below work with the below functions. Others do not. Still working this out.  
#Q12016 = 'https://data.sfgov.org/resource/6iid-qfaz.json'
#Q42015 = 'https://data.sfgov.org/resource/6jnk-ty34.json'
#Q32015 = 'https://data.sfgov.org/resource/8qip-pyye.json'
#Q22015 = 'https://data.sfgov.org/resource/b6nb-tyvq.json'
#Q12015 = 'https://data.sfgov.org/resource/auw5-vpae.json'
#Q42014 = 'https://data.sfgov.org/resource/ia2z-a7eh.json'
#Q32014 = 'https://data.sfgov.org/resource/9xqb-guwy.json'

In [ ]:
def importdata(quarter, field1, field2, field3, field4, field5, field6, field7, field8, field9, geogfield1, geogfield2):
    
    def includekey(field):
        list = []
        for item in data: 
            if field in item.keys():
                list.append(item[field])
            else:
                list.append(np.nan)
        return list

    def include_coor_key(one, two):
        list = []
        for item in data: 
            if field1 in item.keys():
                list.append(item[one][two])
            else:
                list.append(np.nan)
        return list
    
    response = requests.get(quarter)
    results = response.text
    data = json.loads(results) #data is a list at this point
    
    #import fields
    d = {}
    d['lot_number'] = includekey(field1)
    d['address'] = includekey(field2)
    d['status'] = includekey(field3)
    d['latest_date'] = includekey(field4)
    d['units'] = includekey(field5)
    d['net_units'] = includekey(field6)
    d['affordable_units'] = includekey(field7)
    d['net_affordable_units'] = includekey(field8)
    d['zone'] = includekey(field9)
    d['lat_lon'] = include_coor_key(geogfield1, geogfield2)
    
    df = pd.DataFrame.from_dict(d)
    
    return df

In [ ]:
Q22016df = importdata(Q22016, 'apn', 'nameaddr', 'beststat', 'bestdate', 'units', 'unitsnet', 'aff', 'affnet', 'zoning_sim', 'location', 'coordinates')   
#Q12016df = importdata(Q12016, 'apn', 'nameaddr', 'beststat', 'bestdate', 'units', 'unitsnet', 'affordable', 'affordable_net', 'zoning', 'location', 'coordinates')
#Q42015df = importdata(Q42015, 'apn', 'nameaddr', 'beststat', 'bestdate', 'units', 'unitsnet', 'affordable', 'affordab_1', 'zoning_sim', 'geography', 'coordinates')
#Q32015df = importdata(Q32015, 'apn', 'nameaddr', 'beststat', 'bestdate', 'units', 'unitsnet', 'affordable', 'affordablenet', 'zoning_sim', 'location', 'coordinates')
#Q22015df = importdata(Q22015, 'apn', 'nameaddr', 'beststat', 'bestdate', 'units', 'unitsnet', 'affordable', 'affordablenet', 'zoning_sim', 'geography', 'coordinates')
#Q12015df = importdata(Q12015, 'blklot', 'nameaddr', 'beststat', 'bestdate', 'units', 'unitsnet', 'affordable', 'affnet', 'zoning_sim', 'geography', 'coordinates')
#Q42014df = importdata(Q42014, 'blklot', 'nameaddr', 'beststat', 'bestdate', 'units', 'unitsnet', 'affordable', 'affnet', 'zoning_sim', 'geography', 'coordinates')
#Q32014df = importdata(Q32014, 'blocklot', 'location_1_address', 'best_stat', 'bestdate', 'units', 'net_added_units', 'affordable', 'affnet', 'zoning', 'location_1', 'coordinates')


In [ ]:
Q22016df.head()


In [ ]:
#Clean data after importing
Q22016df['lon'] = Q22016df['lat_lon'].astype(str).str.split(',').str[0].str.strip('[')
Q22016df['lat'] = Q22016df['lat_lon'].astype(str).str.split(',').str[1].str.strip(']')
Q22016df['net_units'] = Q22016df['net_units'].astype(int) #convert to integer
Q22016df['units'] = Q22016df['units'].astype(int) #convert to integer
#filter out those observations that have no impact on residential construction (0 net units and 0 units)
Q22016df = Q22016df[(Q22016df['units'] != 0) | (Q22016df['net_units'] != 0)]

In [ ]:
#Explore the data to figure out buckets to map in carto
Q22016df[Q22016df['net_units'] > 0].describe(percentiles = [.1, .2, .3, .4, .5, .6, .7, .8, .9])

In [ ]:
export_path = "Output/current_dev.csv"
Q22016df.to_csv(export_path)

In [ ]:

export_path = "Output/current_dev.csv"
Q22016df.to_csv(export_path)

In [ ]:
#Blog Post: https://www.ocf.berkeley.edu/~bgoggin/2016/10/09/mapping-sf-residential-development/